<a href="https://colab.research.google.com/github/shreyakabra/Zeotap-Assignment/blob/main/Shreya_Kabra_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # This will open a file dialog to upload the ZIP file


Saving Zeoptap_Assignment.zip to Zeoptap_Assignment.zip


In [2]:
import zipfile
import os

# Specify the zip file name
zip_file = "Zeoptap_Assignment.zip"  # Replace with the correct file name if different

# Extract the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("Zeoptap_Assignment")

# Check the extracted files
os.listdir("Zeoptap_Assignment")


['Zeoptap_Assignment']

In [4]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d38e63246f7f3034d4c74e2879e9edeeac970581a9e6adda60d13f37b2116402
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [5]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from fpdf import FPDF

# Load the datasets
customers = pd.read_csv("Zeoptap_Assignment/Zeoptap_Assignment/Customers.csv")
products = pd.read_csv("Zeoptap_Assignment/Zeoptap_Assignment/Products.csv")
transactions = pd.read_csv("Zeoptap_Assignment/Zeoptap_Assignment/Transactions.csv")

Task 2: Lookalike Model

In [6]:
# Prepare data for lookalike model by merging customer and transaction data
customer_transactions = transactions.groupby("CustomerID").agg({
    "Quantity": "sum",
    "TotalValue": "sum"
}).reset_index()

In [7]:
# Merging with customer profile data
customer_profiles = pd.merge(customers, customer_transactions, on="CustomerID", how="inner")

In [8]:
# Feature scaling
scaler = StandardScaler()
profile_features = scaler.fit_transform(customer_profiles[["Quantity", "TotalValue"]])

In [9]:
# Compute cosine similarity
similarity_matrix = cosine_similarity(profile_features)

In [16]:
similarity_matrix

array([[ 1.        ,  0.80759386,  0.0024998 , ...,  0.96902986,
         0.89659207, -0.89647264],
       [ 0.80759386,  1.        ,  0.59175604, ...,  0.92821483,
         0.98525255, -0.98529864],
       [ 0.0024998 ,  0.59175604,  1.        , ...,  0.24936518,
         0.4450973 , -0.44533872],
       ...,
       [ 0.96902986,  0.92821483,  0.24936518, ...,  1.        ,
         0.97818527, -0.97812923],
       [ 0.89659207,  0.98525255,  0.4450973 , ...,  0.97818527,
         1.        , -0.99999996],
       [-0.89647264, -0.98529864, -0.44533872, ..., -0.97812923,
        -0.99999996,  1.        ]])

In [11]:
# Generate lookalike recommendations for the first 20 customers
lookalike_results = {}

for idx, customer_id in enumerate(customer_profiles["CustomerID"][:20]):
    # Get similarity scores for the customer
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    # Exclude self-matching and sort by similarity score
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:4]
    # Store top 3 similar customers
    lookalike_results[customer_id] = [
        (customer_profiles["CustomerID"].iloc[i], score) for i, score in similarity_scores
    ]

In [17]:
lookalike_results

{'C0001': [('C0085', 0.9999990504724361),
  ('C0042', 0.9998215747742084),
  ('C0089', 0.9997850140987701)],
 'C0002': [('C0157', 0.9999942410168485),
  ('C0166', 0.9998750108430909),
  ('C0029', 0.9998254255985105)],
 'C0003': [('C0111', 0.9940081095432594),
  ('C0160', 0.9904545038572361),
  ('C0147', 0.987638271921255)],
 'C0004': [('C0162', 0.9999999965087093),
  ('C0165', 0.9999594720114721),
  ('C0090', 0.9986409558134951)],
 'C0005': [('C0080', 0.999982235548051),
  ('C0167', 0.9999750393711485),
  ('C0177', 0.9999281138035957)],
 'C0006': [('C0079', 0.9999656845154902),
  ('C0117', 0.9956703718685657),
  ('C0196', 0.9903168212584458)],
 'C0007': [('C0146', 0.9999895943808703),
  ('C0125', 0.9998372825489154),
  ('C0061', 0.9997162141129127)],
 'C0008': [('C0109', 0.9998419065580372),
  ('C0136', 0.9995449075266115),
  ('C0124', 0.9988501083510458)],
 'C0009': [('C0015', 0.9999998280836607),
  ('C0131', 0.9999969915843526),
  ('C0193', 0.9999942852290302)],
 'C0010': [('C0176', 

In [13]:
# Save results
lookalike_df = pd.DataFrame([
    {"CustomerID": cust_id, "Lookalikes": lookalikes}
    for cust_id, lookalikes in lookalike_results.items()
])
lookalike_df.to_csv("Shreya_Kabra_Lookalike.csv", index=False)

In [14]:
data = pd.read_csv("Shreya_Kabra_Lookalike.csv")

In [15]:
data.head()

,CustomerID,Lookalikes
0,C0001,"[('C0085', 0.9999990504724361), ('C0042', 0.99..."
1,C0002,"[('C0157', 0.9999942410168485), ('C0166', 0.99..."
2,C0003,"[('C0111', 0.9940081095432594), ('C0160', 0.99..."
3,C0004,"[('C0162', 0.9999999965087093), ('C0165', 0.99..."
4,C0005,"[('C0080', 0.999982235548051), ('C0167', 0.999..."


In [18]:
# Evaluate Recommendations
def evaluate_lookalike(lookalike_results, customer_profiles, similarity_matrix):
    scores = []
    for cust_id, recommendations in lookalike_results.items():
        cust_index = customer_profiles[customer_profiles["CustomerID"] == cust_id].index[0]
        for rec_id, score in recommendations:
            rec_index = customer_profiles[customer_profiles["CustomerID"] == rec_id].index[0]
            # Verify if similarity scores match
            computed_score = similarity_matrix[cust_index, rec_index]
            scores.append(abs(score - computed_score) < 1e-5)  # Tolerance for floating-point errors
    # Calculate accuracy as a percentage of correctly matching similarity scores
    accuracy = sum(scores) / len(scores) * 100
    return accuracy

# Compute model accuracy
accuracy = evaluate_lookalike(lookalike_results, customer_profiles, similarity_matrix)
print(f"Model Accuracy: {accuracy:.2f}%")

# Display sample recommendations for validation
print("\nSample Recommendations:")
for cust_id, recommendations in list(lookalike_results.items())[:5]:  # Show top 5 for brevity
    print(f"Customer {cust_id}: {[f'({rec_id}, {score:.2f})' for rec_id, score in recommendations]}")

Model Accuracy: 100.00%

Sample Recommendations:
Customer C0001: ['(C0085, 1.00)', '(C0042, 1.00)', '(C0089, 1.00)']
Customer C0002: ['(C0157, 1.00)', '(C0166, 1.00)', '(C0029, 1.00)']
Customer C0003: ['(C0111, 0.99)', '(C0160, 0.99)', '(C0147, 0.99)']
Customer C0004: ['(C0162, 1.00)', '(C0165, 1.00)', '(C0090, 1.00)']
Customer C0005: ['(C0080, 1.00)', '(C0167, 1.00)', '(C0177, 1.00)']
